<a href="https://colab.research.google.com/github/pietrzakkuba/computer-assisted-translation-labs/blob/main/lab_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Komputerowe wspomaganie tłumaczenia

# Zajęcia 6 - wykorzystanie tłumaczenia automatycznego we wspomaganiu tłumaczenia

W dzisiejszych czasach, niezwykle ważną techniką wspomagania tłumaczenia jest użycie tłumaczenia maszynowego. Tekst źródłowy do tłumaczenia jest najpierw tłumaczony całkowicie autommatycznie, a następnie tłumacz ludzki dokonuje korekty wyniku. Technologia tłumaczenia maszynowego jest już na tyle dojrzała, że oferuje bardzo wysoką jakość wyników. Coraz częstsze stają się scenariusze, w których ludzka korekta to niemal całkowicie machinalne (sic!) zatwierdzanie wyników tłumaczenia maszynowego. Na dzisiejszych zajęciach poznamy techniki ewaluacji tłumaczenia maszynowego oraz sprawdzania jego przydatności w procesie wspomagania tłumaczenia ludzkiego.

Jakość tłumaczenia maszynowego możemy oceniać na dwóch niezależnych płaszczyznach: dokładność i płynność. Płynność jest subiektywnie odbieranym odczuciem, że czytany tekst jest napisany językiem naturalnym i zrozumiałym. Systemy tłumaczenia maszynowego oparte na uczeniu głębokim z wykorzystaniem sieci neuronowych osiągają duży stopień płynności tłumaczenia. Niestety jednak ich dokładność nie zawsze jest równie wysoka.

Dokładność tłumaczenia maszynowego jest parametrem, który łatwiej zmierzyć. Wartość takich pomiarów daje obraz tego, jaka jest faktyczna jakość tłumaczenia maszynowego i jaka jest jego potencjalna przydatność we wspomaganiu tłumaczenia.

Najczęściej stosowaną techniką oceny tłumaczenia maszynowego jest ocena BLEU. Do obliczenia tej oceny potrzebny jest wynik tłumaczenia maszynowego oraz referencyjne tłumaczenie ludzkie wysokiej jakości.

### Ćwiczenie 1: Zaimplementuj program do obliczania oceny BLEU dla korpusu w folderze data. Użyj implementacji BLEU z pakietu nltk. Dodatkowe wymaganie techniczne - napisz program tak, aby nie musiał rozpakwowywać pliku zip z korpusem na dysku.

In [ ]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize, sent_tokenize
import zipfile
from statistics import mean


def calculate_bleu():
    with zipfile.ZipFile('corpus_corrected.zip', 'r') as archive:
        de_human = sent_tokenize(archive.read('corpus_de_human.txt').decode('utf-8'))
        de_nmt = sent_tokenize(archive.read('corpus_de_nmt.txt').decode('utf-8'))
        scores = []
        smoothie = SmoothingFunction().method7
        for sent_human, sent_nmt in zip(de_human, de_nmt):
            words_human = word_tokenize(sent_human)
            words_nmt = word_tokenize(sent_nmt)
            try:
                scores.append(sentence_bleu([words_human], words_nmt, smoothing_function=smoothie))
            except:
                scores.append(0.5)
    return mean(scores)

calculate_bleu()

0.291773144016802

### Ćwiczenie 2: Oblicz wartość bleu na różnych fragmentach przykładowego korpusu (np. na pierwszych 100 zdaniach, zdaniach 500-600). Czy w jakimś fragmencie korpusu jakość tłumaczenia znacząco odbiega od średniej?

In [ ]:
def analyze_bleu():
    with zipfile.ZipFile('corpus_corrected.zip', 'r') as archive:
        de_human = sent_tokenize(archive.read('corpus_de_human.txt').decode('utf-8'))
        de_nmt = sent_tokenize(archive.read('corpus_de_nmt.txt').decode('utf-8'))
        scores = []
        scores_means = []
        smoothie = SmoothingFunction().method7
        for index, (sent_human, sent_nmt) in enumerate(zip(de_human, de_nmt)):
            words_human = word_tokenize(sent_human)
            words_nmt = word_tokenize(sent_nmt)
            try:
                scores.append(sentence_bleu([words_human], words_nmt, smoothing_function=smoothie))
            except:
                scores.append(0.5)
            if index % 100 == 99:
                scores_means.append(mean(scores))
                scores = []
    return scores_means

analyze_bleu()

[0.7101945871701627,
 0.24108480188163067,
 0.25832618736503127,
 0.24038233553178087,
 0.25185519668555506,
 0.231676401726062,
 0.25200237796517844,
 0.2490376106087679,
 0.24146238507137413,
 0.25557501723145853]

Inną metodą oceny jakości tłumaczenia maszynowego jest parametr WER - Word Error Rate. Definiuje się on w następujący sposób:

$WER = \frac{S+D+I}{N}=\frac{S+D+I}{S+D+C}$

gdzie:
 * S - liczba substytucji (słów)
 * D - liczba usunięć
 * I - liczba wstawień
 * C - liczba poprawnych śłów
 * N - liczba słów w tłumaczeniu referencyjnym (N=S+D+C)

Miara ta jest zwykle używana w do oceny systemów automatycznego rozpoznawania mowy, jednak w kontekście wspomagania tłumaczenia może być rozumiana jako wielkość nakładu pracy tłumacza nad poprawieniem tłumaczenia maszynowego.

### Ćwiczenie 3: Oblicz wartość WER dla przykładowego korpusu. Skorzystaj z gotowej implementacji WER.

In [ ]:
from jiwer import wer

def calculate_wer():
    with zipfile.ZipFile('corpus_corrected.zip', 'r') as archive:
        de_human = sent_tokenize(archive.read('corpus_de_human.txt').decode('utf-8'))
        de_nmt = sent_tokenize(archive.read('corpus_de_nmt.txt').decode('utf-8'))
        scores = []
        for sent_human, sent_nmt in zip(de_human, de_nmt):
            scores.append(wer(sent_human, sent_nmt))
    return sum(scores)

calculate_wer()

1300.3810985454709

Poza wymienionymi powyżej, stosować można jeszcze inne miary oparte na porównywaniu tłumaczenia maszynowego z ludzkim. Przypomnijmy sobie jedną, którą stosowaliśmy wcześniej.

### Ćwiczenie 4: Oblicz średnią wartość dystansu Levenshteina pomiędzy zdaniami przetłumaczonymi automatycznie oraz przez człowieka. Użyj implementacji z ćwiczeń nr 2.

In [ ]:
from Levenshtein import distance as levenshtein_distance

def calculate_levenshtein():
    with zipfile.ZipFile('corpus_corrected.zip', 'r') as archive:
        de_human = sent_tokenize(archive.read('corpus_de_human.txt').decode('utf-8'))
        de_nmt = sent_tokenize(archive.read('corpus_de_nmt.txt').decode('utf-8'))
        scores = []
        for sent_human, sent_nmt in zip(de_human, de_nmt):
            scores.append(levenshtein_distance(sent_human, sent_nmt))
    return mean(scores)

calculate_levenshtein()

100.65493646138808

A teraz sprawdźmy coś jeszcze. W danych przykładowego korpusu znajduje się także angielski tekst źródłowy. Teoretycznie, dobre tłumaczenie niemieckie powinno zawierać jak najwięcej słów z angielskiego źródła. Wykonajmy najstępujący eksperyment:

### Ćwiczenie 5: Dla każdej trójki zdań z korpusu przykładowego wykonaj następujące kroki:
 * Przetłumacz każde angielskie słowo na niemiecki przy użyciu modułu PyDictionary.
 * Sprawdź, które z niemieckich tłumaczeń zawiera więcej spośród tych przetłumaczonych słów - automatyczne, czy ludzkie.
Następnie wypisz statystyki zbiorcze. Które tłumaczenie zawiera więcej słownikowych tłumaczeń słów ze źródła?

In [ ]:
from PyDictionary import PyDictionary

def analyze_translations():
    dictionary = PyDictionary()
    with zipfile.ZipFile('corpus_corrected.zip', 'r') as archive:
        en = sent_tokenize(archive.read('corpus_en.txt').decode('utf-8'))
        de_human = sent_tokenize(archive.read('corpus_de_human.txt').decode('utf-8'))
        de_nmt = sent_tokenize(archive.read('corpus_de_nmt.txt').decode('utf-8'))
        results = []
        for sent_en, sent_human, sent_nmt in zip(en, de_human, de_nmt):
            words_en = word_tokenize(sent_en)
            translated_words = [dictionary.translate(word, 'de') for word in words_en]
            words_human = word_tokenize(sent_human)
            words_nmt = word_tokenize(sent_nmt)
            same_words_human = len(list(set(translated_words).intersection(words_human)))
            same_wprds_nmt = len(list(set(translated_words).intersection(words_nmt)))
            results.append((same_words_human, same_wprds_nmt))
    return results

analyze_translations()

# dla każdego słowa zwraca Invalid Word :(

Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
Invalid Word
